# Starting simple: Linear Regression
As so often, we will start relatively simple with a Linear Regression. Sounds maybe a bit boring but don't worry, I will show you later how to extend this model to something slightly more complex.

## Short Look at the Data
For this tutorial, I will use rental data that I scraped from Immoscout24. For a more detailed description and information on how I scraped the data set, you can check its description on [kaggle](https://www.kaggle.com/corrieaar/apartment-rental-offers-in-germany) where I occasionally also update the data.
For this analysis, we will concentrate on rental offers in Berlin but of course feel free to try out different cities or areas!

In [ ]:
import sys
sys.path.append('../src/')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

from utils import iqr, iqr_rule

In [ ]:
plt.style.use("fivethirtyeight")   # or your own favorite style
plt.rcParams['figure.figsize'] = 9, 9

In [ ]:
d = pd.read_csv("../data/immo_data.csv", dtype={"geo_plz": str})

Before using the data, we will do a bit of preprocessing: We remove outliers where either the living area or the total rent is too low or too high. To remove outliers, the [Inter quartile range (IQR)](https://en.wikipedia.org/wiki/Interquartile_range#Outliers) is used. The IQR rule marks everything as outlier that's too far from the middle range of the data. Most of the data we throw away this way were typos or similar with unreasonable input values.
For a more thorough analysis, it might be useful to check that we don't throw away real cases and instead incorporate the outliers in further analysis.

As target variable we will use the variable `totalRent`, which in most cases is the sum of the base rent and a service fee.

In [ ]:
d["totalRent"] = np.where(d["totalRent"].isnull(), d["baseRent"], d["totalRent"])

# since log doesn't work with 0, we replace 0 with 0.5
# seems reasonable tto say hat a rent of 0€ is the same as 50ct
d["livingSpace_m"] =  np.where(d["livingSpace"] <= 0, 0.5, d["livingSpace"])
d["totalRent_m"] = np.where(d["totalRent"] <= 0, 0.5, d["totalRent"])
d["logRent"] = np.log(d["totalRent_m"])
d["logSpace"] = np.log(d["livingSpace_m"])

not_outlier = iqr_rule(d["logSpace"], factor=1.5) & iqr_rule(d["logRent"], factor=1.5)
d = d[not_outlier]
berlin = d[(d.regio1 == "Berlin") ]

In this analysis, we want to predict the rent (`totalRent`) by the living area (`livingSpace`).

You can have a short look at the data and these two variables!

For example:
- What is the average rent in Berlin?
- What is the average size of a flat in Berlin?
- Plot rent vs living space

In [ ]:
berlin.shape

In [ ]:
berlin.iloc[0]

In [ ]:
# average rent
berlin["totalRent"].mean()

In [ ]:
berlin["totalRent"].median()

The difference in mean and median suggests that the distribution is not normally distributed. If you plot the histogram, you can also see that the distribution is skewed.

In [ ]:
# average living area
berlin["livingSpace"].mean()

In [ ]:
berlin["livingSpace"].median()

In [ ]:
berlin.date.value_counts().plot.bar()
plt.title("Number of cases per scrape time")
plt.show()

In [ ]:
berlin[["totalRent", "livingSpace", "noParkSpaces", "picturecount", "yearConstructed"]].corr()

Living space is strongly positively correlated with the total rents, as we would expect. Interestingly, the construction year doesn't seem to be strongly correlated with total rent, this might be because it is not a linear relationship: Altbau tends to be very expensive in Berlin, Plattenbau from the 70s less so and flats in new buildings are expensive again. Picture count is also positively correlated with the total rent but is a good example that correlation does not imply causation.

In [ ]:
berlin["livingSpace"].mean()

In [ ]:
berlin["totalRent"].mean()

In [ ]:
berlin["totalRent"].median()

In [ ]:
plt.figure(figsize=(9,9))
plt.scatter(berlin.livingSpace, berlin.totalRent, s=4)
plt.title("Rent by living area")
plt.xlabel("Living Area [sqm]")
plt.ylabel("Monthly Rent [€]")
plt.show()

Before working with the data, we will rescale and normalize the living area and also rescale the total rent:

In [ ]:
berlin["livingSpace_s"] = (berlin["livingSpace"] - berlin["livingSpace"].mean()) / np.std(berlin["livingSpace"])
berlin["totalRent_s"] = berlin["totalRent"] / 100

# saving for later
berlin.to_csv("../data/berlin.csv", encoding="utf-8", quoting=csv.QUOTE_NONNUMERIC) # special quoting necessary because otherwise description messes up

We will have to standardize/destandardize area a few times, so we will use small helper functions for this:

In [ ]:
def standardize_area(x):
    return ( x - berlin["livingSpace"].mean()) / np.std(berlin["livingSpace"])
    
def destandardize_area(x):
    return ( x * np.std(berlin["livingSpace"]) ) + berlin["livingSpace"].mean()    

In [ ]:
plt.figure(figsize=(9,9))
plt.scatter(berlin.livingSpace_s, berlin.totalRent_s, s=4)
plt.title("Rent by living area")
plt.xlabel("Living Area [sqm]")
plt.ylabel("Monthly Rent [100€]")
plt.axvline(x=0, c="k", linewidth=1)
plt.axhline(y=np.mean(berlin.totalRent_s), c="k", linewidth=1, linestyle="--", label="Average Rent")
plt.legend()
plt.show()

Looking at the plot, it roughly looks like a linear relationship between area and monthly rent. Indeed, the bigger the flat, the more expensive it should be. So we will start our modelling with a linear model.

So in math notation, our model can be written as follows:

$$ \text{rent} \approx \alpha + \beta \text{area} $$

This is the same as
$$ \text{rent} = \alpha + \beta \text{area} + \epsilon$$
where $\epsilon$ is normally distributed, i.e. $\epsilon \sim \text{Normal}(0, \sigma)$. This can be rewritten as
$$ \text{rent} \sim \text{Normal}(\alpha + \beta \text{area}, \sigma).$$

For easier reading, we rewrite this again:
$$\begin{align*} \text{rent} &\sim \text{Normal}(\mu, \sigma) \\
\mu &= \alpha + \beta \text{area}
\end{align*}$$
This will be our first model!

## Adding a Bayesian ingredient: Priors

Before implementing this model, let's shortly think about what the parameters $\alpha$ and $\beta$ mean here.

We will use the model with our rescaled data and thus $\alpha$ the intercept is the rental price of an average sized flat. (For averaged sized flats, the scaled area is 0).

$\beta$ is then the increase in rent if the flat is one standard deviation larger. One standard deviation is 29sqm which is roughly the average size for a room (check living space divided by number of rooms _noRooms_). Thus $\beta$ is roughly the increase in rent if the flat would have one more room. 

$\sigma$ is how much the rent can differ for two flats of the same size. Concretely, it is how much the rent can differ from the average rent for flats of the same size. As our model says that the rent is normally distributed, about 95% of the cases should be within $2\sigma$ of the average rent.
As error term, $\sigma$ is always positive.

Thinking about what the parameters mean beforehand is very important in a Bayesian analysis since we need to specify priors. Priors are what we think the parameters could be before seeing the data. And, obviously, to be able to say what range the parameters would be in, it would be good to know what the parameters mean.


If we don't know anything about the problems, we might want to specify priors that are very uninformative and vague. 
We could for example specify $\alpha$ and $\beta$ as being somewhere between -10,000 and + 10,000:

In [ ]:
import pymc3 as pm

In [ ]:
with pm.Model() as mod:
    alpha = pm.Normal('alpha', mu=0, sigma=10000)
    beta = pm.Normal('beta', mu=0, sigma=10000)

A PyMC-Model is specified in a context. Before we can actually specify the model, we need to specify the priors, since, as usual in Python, each variable we want to use need to declared beforehand.
In PyMC, you always need to specify the name of the variable twice. This is so that the variable knows its own name.

If you print the model, it renders nicely in a more mathy looking description:

In [ ]:
mod

We can add to the model by opening the context again.
To for example add a prior for sigma, we can proceed as follows:

In [ ]:
with mod:
    sigma = pm.HalfNormal('sigma', sigma=10000)

Since $\sigma$ as error term is always a positive variable, we need to use a distribution that is also always positive. One commonly used distribution for this is the Half-Normal. A normal distribution that is cut in half and only positive.
Other commonly used distributions for $\sigma$ are for example the Exponential or the Half-Cauchy.

Now that we specified some priors, we can write out the complete model:

In [ ]:
with mod:
    mu = alpha + beta*berlin["livingSpace_s"]
    
    rent = pm.Normal('rent',mu=mu, sigma=sigma,
                    observed=berlin["totalRent_s"])

The PyMC-Model is written very similar to how the model was specified above. 

Note that for the outcome variable, we need to specify the observed data.
Usually the whole model is written as one:

In [ ]:
with pm.Model() as mod:
    alpha = pm.Normal("alpha", mu=0, sigma=10000)
    beta = pm.Normal("beta", mu=0, sigma=10000)
    sigma = pm.HalfNormal("sigma", sigma=10000)
    
    mu = alpha + beta*berlin["livingSpace_s"] 
    rent = pm.Normal("rent", mu=mu, sigma=sigma,
                    observed=berlin["totalRent_s"])

Especially in the beginning, when starting out with Bayesian modelling, picking priors can seem a bit daring. 
Does it make a difference if we use $\text{Normal}(0,100)$ or $\text{Normal}(0, 1000)$?
What's with using different distributions?


There are a few tips that help a bit with picking a good prior. The first one is to visualize your priors. We can do this with PyMC by sampling from our priors.
This is the similar to sampling from the specified distributions using numpy or scipy. However, on top of sampling from the probability distributions, it then also computes $\mu$ (the linear part of the model) using the samples and the predictor variables. It then uses the computed $\mu$ and the samples from $\sigma$ to sample possible outcome values. Even though we specified the target variable (in Machine Learning this one is usually called `y`) it does not use this (yet) and is thus very quick.

In [ ]:
with mod:
    priors = pm.sample_prior_predictive(samples=1000)

We will use [ArviZ](https://arviz-devs.github.io/arviz/) to keep track of the different artifacts computed from our model and to visualize them.

In [ ]:
import arviz as az
pm_data = az.from_pymc3(prior = priors)

ArviZ comes with many plots that are useful to analyze Bayesian models.
Let's start looking at the priors for our three model parameters:

In [ ]:
az.plot_density(pm_data, 
                # we want to plot the prior
                group="prior", 
                # and only the following variables
                var_names=["alpha", "beta", "sigma"],
                # just some settings to make it prettier
               shade=0.3, bw=8, figsize=(20,6), credible_interval=1)
plt.show()

This doesn't look super interesting, it is basically just a density plot of the distributions we gave for the priors.
More interesting is to visualize what rental prices these prior distributions would produce given the predictor data.
Unfortunately, there is no ArviZ plot for this yet, but we can do this ourselves without too much work.

The object `priors` contains a numpy array for `rent`. Check what it contains:

In [ ]:
priors["rent"].shape

For each observation in our dataframe (7092 obs) it computed 1000 samples for possible rent prices, using the samples from `alpha`, `beta`, and `sigma` together with the corresponding living area from this observation.

We can flatten the matrix to obtain one big array and plot a histogram:

In [ ]:
plt.hist(priors["rent"].flatten()*100, alpha=0.9, ec="darkblue", bins=70)
plt.title("Histogram over possible range of rental prices")
plt.xlabel("Monthly Rent [€]")
plt.show()

Compare this with the histogram over the actual rental prices:

In [ ]:
plt.hist(berlin["totalRent_s"]*100, alpha=0.9, ec="darkblue", bins=70)
plt.title("Histogram over the actual rental prices")
plt.xlabel("Monthly Rent [€]")
plt.show()

The histograms don't look very similar and the range of the sampled rents is completly off! A rent of up to 50,000,000€ per month doesn't sound very realistic.


Another good way to understand the prior better is to visualize the model that it would produce. In our case, the model is a line determined by the intercept $\alpha$ and the slope $\beta$. 
We can thus sample 50 $\alpha$ and $\beta$s and multiply this with the range of area values:

In [ ]:
area_s = np.linspace(start=-2, stop=3.5, num=50)
draws = np.random.choice(len(priors["alpha"]), replace=False, size=50)
alpha = priors["alpha"][draws]
beta = priors["beta"][draws]

mu = alpha + beta * area_s[:, None]


plt.plot(destandardize_area(area_s), mu*100, c="#737373", alpha=0.5)
plt.xlabel("Living Area [sqm]")
plt.ylabel("Price [€]")
plt.title("Linear model according to our prior")
plt.axhline(y=900, color="#fc4f30", label="My own rent")
plt.axhline(y=5000, color='#e5ae38', label="Most expensive realistic rent I can think of")
plt.axhline(y=0, label="Free rent")
plt.legend()
plt.show()

Exercise: Use `plt.axhline(y=my_rent)` to compare the sampled lines with your own rent, the most expensive, reasonable rent you can think off and the cheapest possible rent you can think off. I also like to google what is the known most expensive rent one can find in Berlin (or elsewhere) and see how it compares to the model.

You could also add these benchmarks to the histogram using `plt.axvline(x=my_rent)`.

These priors are obviously not very realistic! Luckily, we all know a bit about rents, so even without looking at the data, we can think of better priors.

Write a new model `mod_informed` that uses the same linear part as above, but better priors. You can use the same distributions as above and only change its parameters. I recommend you to try various priors and check what effects they have on the resulting model and its outputs.

Remember: 
- $\alpha$ the intercept is the rental price of an average sized flat (averaged sized flat is 77sqm).
- $\beta$ the slope is roughly the increase in rent if the flat would have one more room
- $\sigma$ is how much the rent can differ for two flats of the same size. As error term, $\sigma$ is always positive.

Always keep in mind the scale of your data!

In [ ]:
with pm.Model() as mod_informed:
    # these are definitely not the only good priors for this problem
    # for this problem, many priors, even the original ones,
    # work quite fine for this problem
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=5)
    
    sigma = pm.HalfNormal("sigma", sigma=5)
    
    mu = alpha + beta*berlin["livingSpace_s"]
    
    rent = pm.Normal("rent", mu=mu, sigma=sigma,
                    observed=berlin["totalRent_s"])
    
    priors = pm.sample_prior_predictive(samples=1000)

I made functions for the above plot, so you don't have to copy the whole plot code: `compare_hist(priors, berlin)` to plot the two histograms and `draw_models(priors, berlin)` to plot sampled lines from our model:

In [ ]:
from utils import compare_hist, draw_models

compare_hist(priors, berlin)
plt.show()

In [ ]:
draw_models(priors, berlin)
plt.axhline(y=900, color="#fc4f30", label="My own rent")
plt.axhline(y=5000, color='#e5ae38', label="Most expensive realistic rent I can think of")
plt.axhline(y=0, label="Free rent")
plt.legend()
plt.show()

The priors still cover a very wide range of values, even some values that are not very realistic. It still covers negative values, has negative slopes, and goes up to some still relatively extreme values. But that's fine, afterall, the model will still see the data and then learn from that. But this way we managed to tell the model a plausible range of possible values.


Note: It is not good practise to deduce priors from the data we will train on later. So comparing the prior distributions to the original distribution is not something you should do to get the prior distribution close to the data distribution. I only presented the data here to help get an idea what ranges we're talking about. It's usually better to deduce priors from domain knowledge or external knowledge and not from the data itself!

Questions to explore regarding different priors:
- We've seen above what happens when we use overly vague priors, what happens if we use very narrow priors? What problems could arise?

In [ ]:
with pm.Model() as mod_narrow:
    alpha = pm.Normal("alpha", mu=0, sigma=0.5)
    beta = pm.Normal("beta", mu=0, sigma=0.5)
    
    sigma = pm.HalfNormal("sigma", sigma=0.5)
    
    mu = alpha + beta*berlin["livingSpace_s"]
    
    rent = pm.Normal("rent", mu=mu, sigma=sigma,
                    observed=berlin["totalRent_s"])
    
    priors_narrow = pm.sample_prior_predictive(samples=1000)

In [ ]:
compare_hist(priors_narrow, berlin)
plt.show()

This way, the model covers a very small range of values it thinks are plausible and unfortunately, the real range is not included in this range!
Using very (too) narrow priors is the same as using (too) heavy regularization in Machine Learning: The model underfits and cannot learn from the data because we restrict it too strongly. 

In this case, since the normal distribution puts at least a tiny tiny bit of probability mass even on far away values and we have a lot of data, the model still manages to learn the correct values. But we're making it quite difficult for the model! If we'd have less data this would certainly not be the case.

A good source on which priors to pick for which problem is the [Stan Prior Choice Recommendations](https://github.com/stan-dev/stan/wiki/Prior-Choice-Recommendations).

- So far, we've only used the normal distribution for the priors. Try out some different distributions, e.g.
    - Uniform distribution, e.g. over -100 to + 100

In [ ]:
with pm.Model() as mod_uniform:
    alpha = pm.Uniform("alpha", lower=-100, upper=100)
    beta = pm.Uniform("beta", lower=-100, upper=100)
    
    sigma = pm.HalfNormal("sigma", sigma=0.5)
    
    mu = alpha + beta*berlin["livingSpace_s"]
    
    rent = pm.Normal("rent", mu=mu, sigma=sigma,
                    observed=berlin["totalRent_s"])
    
    priors_uniform = pm.sample_prior_predictive(samples=1000)

In [ ]:
compare_hist(priors_uniform, berlin)
plt.show()

The prior predictive distributions looks actually quite similar to the ones we got above. In this case, the results will also look very similar.


However, when we use a uniform prior distribution, we introduce hard borders. Hard borders are generally not a good idea: 
They can make it difficult for the sampling algorithm (using for example $\text{Uniform}(-1000, +1000)$ beaks the sampling!) but they're especially problematic when the true parameter value is not inside this hard border range:

In [ ]:
with pm.Model() as mod_uniform_narrow:
    alpha = pm.Uniform("alpha", lower=-0.5, upper=0.5)
    beta = pm.Uniform("beta", lower=-0.5, upper=0.5)
    
    sigma = pm.HalfNormal("sigma", sigma=0.5)
    
    mu = alpha + beta*berlin["livingSpace_s"]
    
    rent = pm.Normal("rent", mu=mu, sigma=sigma,
                    observed=berlin["totalRent_s"])
    
    trace_uniform_narrow = pm.sample(draws=1000, tune=1000)
    priors_uniform_narrow = pm.sample_prior_predictive(samples=1000)

This model puts zero probability mass on the correct parameter values: It will thus fail and always produce wrong results, no matter how much data we feed. This is not so much visible in the prior predictive distribution (except for it being far off from plausbile values) but shows in the results:

In [ ]:
az.summary(trace_uniform_narrow)

Both alpha and beta are exactly at the border of their prior distribution and if we compare it later to our actual model, we will see that these values are completely off.

So unless you have good reasons for it, I would not recommended to use the Uniform distribution as prior.

    - Student-T distribution
    
The Student-T distribution is often used instead of the Normal distribution for the rent distribution. The t-distribution has much heavier tails which makes it much more robust against outlier. it is thus often also called a robust regression. To use the T-distribution, we will have to estimate another parameter $\nu$ nu, the degrees of freedom.

In [ ]:
with pm.Model() as mod_robust:
    alpha = pm.Normal("alpha", mu=0, sigma=10)
    beta = pm.Normal("beta", mu=0, sigma=5)
    
    sigma = pm.HalfNormal("sigma", sigma=0.5)
    
    mu = alpha + beta*berlin["livingSpace_s"]
    
    # nu, the degrees of freedom is a new parameter
    # this is a commonly used default prior used for nu
    nu = pm.Gamma("nu", alpha=2, beta=0.1)
    
    rent = pm.StudentT("rent", nu=nu, mu=mu, sigma=sigma,
                       observed=berlin["totalRent_s"])
    
    trace_robust = pm.sample(draws=1000, tune=1000)
    priors_robust = pm.sample_prior_predictive(samples=1000)

In [ ]:
az.plot_density(az.from_pymc3(prior=priors_robust, model=mod_robust), 
                group="prior",
                var_names=["alpha", "beta", "sigma", "nu"],
               shade=0.3, credible_interval=1, bw=8)
plt.show()

Since for large values of nu, the T-distribution also approaches a Normal distribution, we can even check how "normal" our data is. In a robust regression, it is even possible to [detect outliers](https://docs.pymc.io/notebooks/GLM-robust-with-outlier-detection.html).

In [ ]:
az.summary(trace_robust)

In this case, $\nu$ is relatively high, indicating that our data is pretty normal. Well, kind of by design since I cleaned out the outliers before.
You can go and try how this changes if you leave the outliers (or at least some outliers) in.

- Since we know that rents should increase for larger flats, we know that the slope $\beta$ should be positive, how could we bias our priors to positive values?

One possibility is to just shift the mean of beta a bit up:

In [ ]:
with pm.Model() as mod_pos_slope:
    alpha = pm.Normal("alpha", mu=5, sigma=10)
    beta = pm.Normal("beta", mu=2, sigma=5)
    
    sigma = pm.HalfNormal("sigma", sigma=5)
    
    mu = alpha + beta*berlin["livingSpace_s"]
    
    rent = pm.Normal("rent", mu=mu, sigma=sigma,
                    observed=berlin["totalRent_s"])
    
    priors_pos_slope = pm.sample_prior_predictive(samples=1000)

In [ ]:
draw_models(priors_pos_slope, berlin)
plt.show()

This way, there are still a few negative slope lines but most have now a positive slope.

Since there is quite a lot of data, in this problem it doesn't make too much of a difference though to use this slightly adapted prior.


Another option is to consider a transformation of the data: For both the living area and the rental price, it might make sense to log-transform these variables.

Optional exercises for at home: 
To understand better how something works, it often helps to implement it oneself. Try implementing yourself sampling from the prior using only numpy or scipy.

In [ ]:
# implementing prior sampling
n = 1000
alpha = np.random.normal(loc=0, scale=10, size=n)
beta = np.random.normal(loc=0, scale=5, size=n)

sigma = np.random.normal(loc=0, scale=5, size=n)
sigma = np.absolute(sigma)

mu = alpha + beta*berlin["livingSpace_s"][:,None]

rent = np.random.normal(loc=mu, scale=sigma).reshape(1,n,len(berlin))

In [ ]:
own_data = az.from_dict(prior = {"alpha":alpha,
                                 "beta":beta,
                                 "sigma":sigma},
            prior_predictive={"rent":rent})

In [ ]:
az.plot_density(own_data, group="prior",
               shade=0.3, bw=8, credible_interval=1)
plt.show()

In [ ]:
prior = {"alpha":alpha,
         "beta":beta,
         "sigma":sigma,
         "rent": rent}
compare_hist(prior, berlin)
plt.show()

In [ ]:
draw_models(prior, berlin)
plt.show()